In [9]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [2]:
import gym 
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

/opt/miniconda3/envs/cs234-torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Load Environment

In [3]:
environment_name = "CartPole-v0"

In [4]:
env = gym.make(environment_name)

In [13]:
env.state

In [15]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        # action_space indicate the number of actions in the environment
        action = env.action_space.sample()
        # .step(action) returns the next state, reward, done, and info according to the action
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

2023-03-22 16:28:12.608 python[10997:5545730] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fabb065a7d0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-03-22 16:28:12.610 python[10997:5545730] Warning: Expected min height of view: (<NSButton: 0x7fabac6e09e0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-03-22 16:28:12.616 python[10997:5545730] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7faba641ff40>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-03-22 16:28:12.618 python[10997:5545730] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7fabac6e1690>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


NameError: name 'glPushMatrix' is not defined

: 

# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [5]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [6]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([2.6678188e+00, 3.3180836e+38, 2.0447029e-01, 5.3412083e+37],
      dtype=float32)

# 3. Train an RL Model

In [5]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


In [8]:
# time steps, the number of times the agent interacts with the environment. # of env.step()
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 1886 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1329        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008627245 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000241   |
|    learning_rate        | 0.0003      |
|    loss                 | 7.14        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0178     |
|    value_loss           | 52.8        |
-----------------------------------------
----------------------------------

# 4. Save and Reload Model

In [6]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [13]:
model.save(PPO_path)

/opt/miniconda3/envs/cs234-torch/lib/python3.8/site-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [14]:
del model

In [7]:
model = PPO.load(PPO_path, env=env)

# 4. Evaluation

In [8]:
from stable_baselines3.common.evaluation import evaluate_policy

In [9]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)
# try out 10 episodes and return the mean reward and std of the reward

/opt/miniconda3/envs/cs234-torch/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [10]:
env.close()

# 5. Test Model

In [ ]:
obs = env.reset()
while True:
    # use the model the predict the next action
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

In [ ]:
env.close()

# 6. Viewing Logs in Tensorboard

In [11]:
training_log_path = os.path.join(log_path, 'PPO_3')

NameError: name 'log_path' is not defined

In [ ]:
!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()